# HOMEWORK 5

In this homework you are going to implement the **Floyd-Steinberg dithering** algorithm. Dithering, in general, means that we are adding noise to the signal (in our case digital image) in order to perceive it better. In other words, by adding the noise the objective quality will be worse but the subjective quality will be better (i.e. the image will "look" better).

The details of FS dithering can be found in this [wiki](https://en.wikipedia.org/wiki/Floyd%E2%80%93Steinberg_dithering) page. In order to implement the dithering, we will implement the following steps:
* Define colour pallette
* Quantize the image to obtain the baseline and compute the average quantization error
* Implement FS dithering and compute the average quantization error

You will also have to answer the question at the end of this notebook.

Note: In this homework, you will have the chance to earn some extra points. See the "Bonus" section at the end of the notebook. Good luck!

As always, you are encouraged to use your own images :-)

In [1]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

Let's load the image.

In [ ]:
# Load image
img = cv2.imread('./data/kodim23.png')
# Convert it to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# Plot it
plt.imshow(img);

Let's start with gray tones first.

In [3]:
# Black, dark gray, light gray, white
colors = np.array([[0, 0, 0],
                   [64, 64, 64],
                   [192, 192, 192],
                   [255, 255, 255]])

Using the colour pallette, let's quantize the original image.

In [67]:
import functools as ft


# Prepare for quantization
rows, cols, channels = img.shape

# Apply quantization
def apply_quantization_gray(pixel, palette):
    pixel = pixel.astype(np.float32)
    min_diff = np.inf
    selected_color = None
    
    for c in palette:
        gray = np.sqrt((pixel[0] ** 2 + pixel[1] ** 2 + pixel[2] ** 2) / 3)
        diff_sq = (c[0] - gray) ** 2
        if diff_sq < min_diff:
            min_diff = diff_sq
            selected_color = c

    return selected_color

quantize = ft.partial(apply_quantization_gray, palette=colors)

In [68]:
quantized = np.apply_along_axis(quantize, 2, img)

In [ ]:
# Show quantized image (don't forget to cast back to uint8)
plt.imshow(quantized);

In [70]:
# Compute average quantization error
rows, columns, _ = img.shape
avg_quant_error = (img - quantized).sum() / (rows * columns)

In [71]:
print(avg_quant_error)

16.755650838216145


#### Floyd-Steinberg Dithering
We are now going to implement the FS dithering and compare it to the optimally quantized image we have calculated above.

In [72]:
# Make a temporal copy of the original image, we will need it for error diffusion
img_tmp = np.copy(img)
dithering = np.zeros_like(img)


def mix_error(pixel, error, k):
    return np.clip(pixel + error * k, 0, 255)
    

for r in range(0, rows-1):
    for c in range(1, cols-1):
        # Extract the original pixel value
        pixel = img_tmp[r, c, :]
        # Find the closest colour from the pallette (using absolute value/Euclidean distance)
        # Note: You may need more than one line of code here
        new_pixel = quantize(pixel)
        
        # Compute quantization error
        quant_error = pixel - new_pixel
        
        # Diffuse the quantization error accroding to the FS diffusion matrix
        # Note: You may need more than one line of code here
        img_tmp[r, c + 1] = mix_error(img_tmp[r, c + 1], quant_error, 7 / 16)
        img_tmp[r + 1, c - 1] = mix_error(img_tmp[r + 1, c - 1], quant_error, 1 / 16)
        img_tmp[r + 1, c] = mix_error(img_tmp[r + 1, c], quant_error, 3 / 16)
        img_tmp[r + 1, c + 1] = mix_error(img_tmp[r + 1, c + 1], quant_error, 5 / 16)
        
        # Apply dithering
        dithering[r, c, :] = new_pixel

In [ ]:
# Show quantized image (don't forget to cast back to uint8)
plt.subplot(121), plt.imshow(quantized)   # optimally quantized
plt.subplot(122), plt.imshow(dithering);   # dithering

In [74]:
# Compute average quantization error for dithered image
avg_dith_error = avg_quant_error = (img - dithering).sum() / (rows * columns)

In [75]:
print(avg_dith_error)

363.91185760498047


### Questions
* Which image has higher quantization error? Optimally quantized or dithered?
* Which image looks better to you?
* Can you repeat the same process using only two colours: black and white? Show me :-)

> Which image has higher quantization error? Optimally quantized or dithered?

Dithering image has higher quantization error

> Which image looks better to you?

Dithering image

> Can you repeat the same process using only two colours: black and white? Show me :-)


In [ ]:
def dither_image(img, quantize_fn):
    # Make a temporal copy of the original image, we will need it for error diffusion
    img_tmp = np.copy(img)
    dithering = np.zeros_like(img)
    
    for r in range(0, rows-1):
        for c in range(1, cols-1):
            # Extract the original pixel value
            pixel = img_tmp[r, c]
            # Find the closest colour from the pallette (using absolute value/Euclidean distance)
            # Note: You may need more than one line of code here
            new_pixel = quantize_fn(pixel)
            
            # Compute quantization error
            quant_error = pixel - new_pixel
            
            # Diffuse the quantization error accroding to the FS diffusion matrix
            # Note: You may need more than one line of code here
            if (0 < r < rows - 1) and (0 < c < cols - 1): 
                img_tmp[r, c + 1] = mix_error(img_tmp[r, c + 1], quant_error, 7 / 16)
                img_tmp[r + 1, c - 1] = mix_error(img_tmp[r + 1, c - 1], quant_error, 1 / 16)
                img_tmp[r + 1, c] = mix_error(img_tmp[r + 1, c], quant_error, 3 / 16)
                img_tmp[r + 1, c + 1] = mix_error(img_tmp[r + 1, c + 1], quant_error, 5 / 16)
            
            # Apply dithering
            dithering[r, c] = new_pixel
    return dithering


quantize_bw = ft.partial(apply_quantization, palette=np.array([[0, 0, 0], [255, 255, 255]]))

bw_img = dither_image(img, quantize_bw)
plt.imshow(bw_img);

### Bonus Points

Repeat the homework using a diffrerent image pallette. For instance, you can use an optimal colour
pallette that we can calculate via k-means algorithm. The following snippet of code will give you the 16
optimal colours for your original image.

In [81]:
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=16).fit(np.reshape(img, (-1, 3)))
colors = kmeans.cluster_centers_

In [78]:
def apply_quantization_color(pixel, palette):
    pixel = pixel.astype(np.float32)
    min_diff = np.inf
    selected_color = None
    
    for c in palette:
        diff_sq = (c[0] - pixel[0]) ** 2 + (c[1] - pixel[1]) ** 2 + (c[2] - pixel[2]) ** 2
        if diff_sq < min_diff:
            min_diff = diff_sq
            selected_color = c

    return selected_color

quantize_km = ft.partial(apply_quantization_color, palette=colors)

In [ ]:
plt.imshow(dither_image(img, quantize_km));

Apply FS dithering the same way you did before.
* How does the result look like to you?
* What happens if we use 32 colours?
* And what happens if we use 256 colours?

In [85]:
def dither_km(img, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters).fit(np.reshape(img, (-1, 3)))
    colors = kmeans.cluster_centers_
    
    quantize_km = ft.partial(apply_quantization_color, palette=colors)
    return dither_image(img, quantize_km)

In [ ]:
plt.imshow(dither_km(img, 32));

In [ ]:
plt.imshow(dither_km(img, 256));